In [1]:
import numpy as np
import pandas as pd
from os.path import join
import os
import time
from pylab import rcParams
import matplotlib.pyplot as plt

import nilmtk
from nilmtk import DataSet, TimeFrame, MeterGroup, HDFDataStore
from nilmtk.disaggregate import CombinatorialOptimisation, FHMM, Hart85
from nilmtk.utils import print_dict
from nilmtk.metrics import f1_score

import warnings
warnings.filterwarnings("ignore")

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 30 days


In [2]:
import sys

In [11]:
sys.path.append("../../code/common/")
sys.path.append("../../code/fridge/")

In [12]:
ds = DataSet("/Users/nipunbatra/Downloads/wikienergy-2.h5")
fridges = nilmtk.global_meter_group.select_using_appliances(type='fridge')

In [13]:
fridges_id_building_id = {i:fridges.meters[i].building() for i in range(len(fridges.meters))}

In [14]:
fridge_id_building_id_ser = pd.Series(fridges_id_building_id)

In [15]:
fridge_id_building_id_ser.head()

0      1
1      1
2     10
3    101
4    102
dtype: int64

In [16]:
from fridge_compressor_durations_optimised import compressor_powers, defrost_power

In [17]:
fridge_ids_to_consider = compressor_powers.keys()

In [18]:
len(fridge_ids_to_consider)

96

In [19]:
building_ids_to_consider = fridge_id_building_id_ser[fridge_ids_to_consider]

In [20]:

building_ids_to_consider.head()

1       1
2      10
8     105
11     11
13    112
dtype: int64

In [21]:
out = {}
for f_id, b_id in building_ids_to_consider.iteritems():
    try:
        print "*"*80
        print f_id
        print "*"*80
        start = time.time()
        out[f_id] = {}
        # Need to put it here to ensure that we have a new instance of the algorithm each time
        cls_dict = {"CO": CombinatorialOptimisation(), "FHMM":FHMM(), "Hart":Hart85()}
        #cls_dict = {"FHMM":FHMM()}
        elec = ds.buildings[b_id].elec
        mains = elec.mains()
        elec.appliances
        fridge_instance = fridges.meters[f_id].appliances[0].identifier.instance
        # Dividing train, test
        train_fraction = 0.3
        train = DataSet("/Users/nipunbatra/Downloads/wikienergy-2.h5")
        test = DataSet("/Users/nipunbatra/Downloads/wikienergy-2.h5")
        split_point = elec.train_test_split(train_fraction=train_fraction).date()
        train.set_window(end=split_point)
        test.set_window(start=split_point)
        train_elec = train.buildings[b_id].elec
        test_elec = test.buildings[b_id].elec
        test_mains = test_elec.mains()

        # Fridge elec
        fridge_elec_train = train_elec[('fridge', fridge_instance)]
        fridge_elec_test = test_elec[('fridge', fridge_instance)]
        fridge_elec_mg = MeterGroup([fridge_elec_train])
        fridge_df_train = fridge_elec_train.load().next()[('power','active')]
        fridge_power = fridge_df_train[(fridge_df_train>compressor_powers[f_id][0])&(fridge_df_train<compressor_powers[f_id][1])]
        fridge_compressor_gt = fridge_power.mean()
        out[f_id]["GT"] = fridge_compressor_gt

        for clf_name, clf  in cls_dict.iteritems():

            if clf_name =="Hart":
                clf.train(train_elec.mains())
                d = (clf.centroids-fridge_power).abs()
                fridge_num = d.sort(ascending=True).head(1).index.values[0]
                fridge_identifier_tuple = ('unknown', fridge_num)
                fridge_compressor_algo = clf.centroids.ix[fridge_num][('power', 'active')]
            elif clf_name=="CO":
                clf.train(fridge_elec_mg)
                fridge_compressor_algo = clf.model[0]['states'][1]
            else:
                # FHMM
                clf.train(fridge_elec_mg)
                fridge_compressor_algo = clf.model.means_[1][0]
            out[f_id][clf_name] = fridge_compressor_algo
    except:
        pass

********************************************************************************
1
********************************************************************************
Training model for submeter 'ElecMeter(instance=15, building=1, dataset='WikiEnergy', appliances=[Appliance(type='fridge', instance=2)])'
Finding Edges, please wait ...
Edge detection complete.
Creating transition frame ...
Transition frame created.
Creating states frame ...
States frame created.
Finished.
Training model for submeter 'ElecMeter(instance=15, building=1, dataset='WikiEnergy', appliances=[Appliance(type='fridge', instance=2)])'
Done training!
********************************************************************************
2
********************************************************************************
Training model for submeter 'ElecMeter(instance=7, building=10, dataset='WikiEnergy', appliances=[Appliance(type='fridge', instance=1)])'
Finding Edges, please wait ...
Edge detection complete.
Creating transiti

In [22]:
from sklearn import linear_model

In [23]:
d = pd.DataFrame(out).T.dropna()



In [32]:
d.to_csv("../../data/fridge/fridge_power_level_nilm.csv")

In [24]:
d_new = d[(d["Hart"]-d["GT"]).abs()<100]

In [25]:
import math
def sign(n):
    if n>0.0:
        return "+"
    else:
        return "-"

In [31]:
np.random.seed(42)
from common_functions import latexify, format_axes
latexify(columns=2, fig_height=2.6)
fig, ax = plt.subplots(ncols=3, sharey=True)
for i, algo in enumerate(["CO", "FHMM", "Hart"]):
    x = d_new[["GT"]]
    y = d_new[[algo]]
   

    # Train the model using the training sets
    
    model_ransac = linear_model.RANSACRegressor(linear_model.LinearRegression())
    model_ransac.fit(x, y)
    ax[i].scatter(x, y, color="gray", alpha=0.5)
    
    #ax[i].set_xlabel("Actual energy (kWh)")
    
    ax[i].plot(x, model_ransac.predict(x), color='black')
    format_axes(ax[i])
    mean_average_error = np.mean(np.abs(y-model_ransac.predict(x)))
    title = algo + "\n" + "y = %0.2f x %s %0.2f \n Mean Average Error in Power = %0.1fW" %(model_ransac.estimator_.coef_[0], 
                                                   sign(model_ransac.estimator_.intercept_[0]),
                                                   math.fabs(model_ransac.estimator_.intercept_[0]),
                                                                   mean_average_error)
    #title = title +"\n" + str(scipy.stats.pearsonr(x, y))
    ax[i].set_title(title)
    
    ax[i].set_xlabel("Actual steady state power (W)")
ax[0].set_ylabel("Predicted steady state power (W)")
plt.tight_layout()
plt.savefig("../../figures/fridge/disag_algos_power_level.pdf", bbox_inches="tight")
plt.savefig("../../figures/fridge/disag_algos_power_level.png", bbox_inches="tight")

In [41]:
pd.DataFrame(out).T

,CO,FHMM,GT,Hart
1,99,98.980790,100.716307,118.513791
2,103,117.215331,104.665396,116.953427
